In [2]:
import cx_Oracle 
import pyodbc 
import warnings
import win32com.client as win32
warnings.simplefilter("ignore")
import mysql.connector
from openpyxl import load_workbook
import tkinter as tk
import pandas as pd
from tkinter import *   
from tkinter import messagebox  

# PRODUÇÃO - ORACLE - LOAD PROJETOS

In [3]:
def ConectaOracle():
    global conn
    global cursorORC
    global projetos
    
#    conn = pyodbc.connect('DRIVER={Devart ODBC Driver for Oracle};Direct=True;Host=masrv-scan.agricultura.gov.br;Service Name=prodmapa.agricultura.gov.br;User ID=bismarckhegermann;Password=mapa1234')
    try:
        sconexao = "bismarckhegermann/mapa1234@masrv-scan.agricultura.gov.br/prodmapa.agricultura.gov.br"
        conn = cx_Oracle.connect(sconexao)
        print("Conectado ao servidor ORACLE - PRODUÇÃO")
    except cx_Oracle.DatabaseError as e:
        error, = e.args
        if error.code == 1017:
            print("STRING de Conexão Incorreta")
        else:
            print("VPN não Conectada...Ligue a VPN => ", e)  
            return
            
            
    cursorORC = conn.cursor()
    
    cursorORC.execute("SELECT ID_PROJETO, NM_PROJETO FROM GOVERNANCA_CGTI_MAPA.S_PROJETO ORDER BY ID_PROJETO DESC")
    row = cursorORC.fetchone() 
    projetos = []

    while row: 
        projetos.append(row)
        row = cursorORC.fetchone()
#ConectaOracle()

# CONECTA MYSQL - OTRS - PRODUÇÃO

In [4]:
def ConectaMySql():
    global con
    global cursorMySql
    
    try:
        con = mysql.connector.connect(host='192.168.2.132', database='otrs', port=3306, user='painel_otrs_apl',password='painel_otrs_apl_1234', buffered=True)
    except mysql.connector.Error as err:
        print("Sem VPN...: {}".format(err))
        return
    
    if con.is_connected():
        db_info = con.get_server_info()
        print("Conectado ao servidor MySQL PRODUÇÃO versão ",db_info)
        cursorMySql = con.cursor()
#ConectaMySql()

# BUSCA ID TORRE/FILA NO OTRS

In [5]:
def BuscaTorre(DescTorre):
    cursorMySql.execute("select ID, substring_index(name, '::' , -1) from queue where upper(substring_index(name, '::' , -1)) like" + "'%" + DescTorre.upper() + "%'" + ";")
    IDTorre = ""
    DescTorre = ""
    linha = cursorMySql.fetchone()
    if pd.isnull(linha) == False:
        IDTorre = linha[0]
        DescTorre = linha[1]
    return(IDTorre, DescTorre)
#BuscaTorre("Administração de SO Linux / Storage/Virtualização")

# BUSCA ID DO SERVIÇO NO OTRS 

In [6]:
def BuscaServico(Servico):
    IDServ = ""
    DescServ = ""
    cursorMySql.execute("select ID, substring_index(name, '::' , -1) from service where upper(name) like" + "'%" + Servico.strip().upper() + "%'" + ";")
    linha = cursorMySql.fetchone()
    if pd.isnull(linha) == False:
        IDServ = linha[0]
        DescServ = linha[1]
    return(IDServ, DescServ)
#BuscaServico("Criar / Configurar / Alterar Vcenter Storage Cluster")

# BUSCA UST e SLA

In [7]:
def BuscaSLA(Torre, Servico, Prioridades): 
    
    if Prioridades != "Zero":   
        SQLUST = "SELECT service_id, sla_id, substring_index(substring_index(service.name, '::' , -1) , 1, LENGTH(substring_index(sla.name, '::' , -1))-1) as UST, solution_time, substr(substring_index(sla.name, '::' , 1) , 6, LENGTH(substring_index(sla.name, '::' , -2))) as Complexidade \
                FROM service \
                inner join service_sla on service.id = service_sla.service_id \
                inner join sla on service_sla.sla_id = sla.id \
                where upper(substring_index(substring_index(service.name, '::' , -2), '::' , 1)) like '%" + Torre.strip() + "%' \
                and upper(substring_index(substring_index(service.name, '::' , -2), '::' , -1)) Like '%" + Servico.strip() + "%'  \
                and substr(substring_index(sla.name, '::' , 1) , 6, LENGTH(substring_index(sla.name, '::' , -2))) Like '%" + Prioridades.strip() + "%' \
                and sla.valid_id = 1"
    else:
        print("Entrei")
        SQLUST = "SELECT service_id, sla_id, substring_index(substring_index(service.name, '::' , -1) , 1, LENGTH(substring_index(sla.name, '::' , -1))-1) as UST, solution_time, substr(substring_index(sla.name, '::' , 1) , 6, LENGTH(substring_index(sla.name, '::' , -1))) as Complexidade \
                FROM service \
                inner join service_sla on service.id = service_sla.service_id \
                inner join sla on service_sla.sla_id = sla.id \
                where upper(substring_index(substring_index(service.name, '::' , -2), '::' , 1)) like '%" + Torre.strip() + "%' \
                and upper(substring_index(substring_index(service.name, '::' , -2), '::' , -1)) Like '%" + Servico.strip() + "%'  \
                and sla.valid_id = 1"
        
    cursorMySql.execute(SQLUST)
    UST = cursorMySql.fetchone()
    if pd.isnull(UST) != True:  
        return(UST[0], UST[1], UST[2], UST[3], UST[4])
    else:
        return((0,0, "Ser/Ser_SLA - VALID_ID=2", 0, ""))

#BuscaSLA("Especialista em Redes ( Backup )", "Restaurar / Acompanhar Alta Volumetria ( > 1tb)", "Média")

# TELA DE DIGITAÇÃO DO CAMINHO E ARQUIVO EXCEL 

In [8]:
def TelaMain():
    print("Conectando aos Bancos")
    ConectaMySql()
    ConectaOracle()

    def submitValues(evt):
        global arquivo, projSel, Pastas, var_lst, pastas, xls

        arquivo = entry_1.get()

        projSel = listbox.get(listbox.curselection())
        projSel = projSel.split(" ")
        projSel = int(projSel[0])

        try:
            print("Abrindo o PP...")
            xls = pd.ExcelFile(arquivo)
            Pastas = xls.sheet_names

        except FileNotFoundError as e:
            print("Planilha Não Encontrada..." + str(e))
            messagebox.showerror("ERROR","Planilha Não Encontrada...")  
            return()

        pastas = {Pastas[i]: Pastas[i] for i in range(0, len(Pastas), 2)}

        v = StringVar(frame2, "1")     

        x = 10
        for pasta in Pastas: 
            pastas[pasta] = IntVar()
            Opt1 = Checkbutton(frame2, text=pasta,variable=pastas[pasta], command=Check, onvalue = 1, offvalue = 0 ).place(x = 15, y = 55 + x)
            x += 20 

        mainloop() 

    def Check():
        global Apagar
        Apagar = IntVar()
        Opt2 = Checkbutton(window_main, text = "Apagar Planejamento Anterior ?",variable = Apagar, onvalue = 1, offvalue = 0 ).place(x = 350, y = 250)
        submit = tk.Button(window_main, text = "Submit", width = 25, bd = '5', activebackground = "pink", background = "pink", command=submitPasta, state=tk.ACTIVE).place(x = 350, y = 280)

    def submitPasta():   
        global PastasSel
        PastasSel = []
        for check in Pastas:
            if pastas[check].get() == 1:
                PastasSel.append(check)

        window_main.destroy()
        InsertExcel()

    window_main = tk.Tk(className=' SONDA Tecnologia' )
    window_main.geometry("800x380")

    frame1=Frame(window_main, width=600, height=230)
    frame1.grid(row=0, column=0)
    frame2=Frame(window_main, width=195, height=230)
    frame2.grid(row=0, column=1)

    entry_1 = StringVar(window_main, value='D:/MAPA/Importacao/PP_Sisbravet_Painel_9.xlsx')

    entry_widget_1 = Label(window_main, text = "Caminho e Nome do Arquivo.: ", font = ('calibre',10,'bold')).place(x = 0,y = 2)  
    entry_widget_1 = Entry(window_main, textvariable=entry_1, width = 95, bd=5).place(x = 207, y = 3)  

    lbl = Label(window_main, text = "Selecione o Projeto a Ser Importado. MUITA ATENÇÃO ESCOHA O CORRETO").place(x = 7, y = 35)
    lbl = Label(frame2, text = "Selecione PASTAS Para Importação").place(x = 0, y = 35)  

    listbox = Listbox(frame1)
    listbox.place(x = 7, y = 60, width = 578)
    listbox.bind('<<ListboxSelect>>',submitValues)

    for proj in projetos:
        listbox.insert(END, str(proj).replace("(", "").replace(",", ""))

    window_main.mainloop()

# Inserir Atividade do EXCEL no ORACLE

In [9]:
def InsertExcel():
    if not con.is_connected():
        print("Conectando aos Bancos")
        ConectaMySql()
        ConectaOracle()

    IDAtividade = 0
    cont = 0
    Container = 1
    IdPlanejamento = 1

    if Apagar.get() == 1:
        print("Excluindo Planejamento Anterior Para o Projeto..: ", str(projSel))

        SqlDel = "DELETE FROM governanca_cgti_mapa.s_projeto_planejamento_ativid WHERE id_projeto = " + str(projSel)
        cursorORC.execute(SqlDel)
        conn.commit()

        SqlDel = "DELETE FROM governanca_cgti_mapa.s_projeto_planejamento_fase WHERE id_projeto = " + str(projSel)
        cursorORC.execute(SqlDel)
        conn.commit()

        SqlDel = "DELETE FROM governanca_cgti_mapa.s_projeto_planejamento WHERE id_projeto = " + str(projSel)
        cursorORC.execute(SqlDel)
        conn.commit()

    print("Abrindo o PP Para o Projeto...: ", projSel)
    print("Importando o PP.: ", arquivo)

    cursorORC.execute("SELECT  ID_PROJETO FROM GOVERNANCA_CGTI_MAPA.S_PROJETO_PLANEJAMENTO P WHERE P.ID_PROJETO = " + str(projSel))
    RegPla = cursorORC.fetchone()     

    if pd.isnull(RegPla) == True:  

        InsertPlanejamento = "INSERT INTO governanca_cgti_mapa.s_projeto_planejamento (id_planejamento, id_projeto, ds_login_elaborador) VALUES (" + str(IdPlanejamento) + "," + str(projSel) + ", 'Sistema Automatico')"
        cursorORC.execute(InsertPlanejamento)
        conn.commit()
        print("Registro Inserido Tabelas PLANEJAMENTO Projeto => ", projSel)

        if Container == 0:
            while cont <= 5:
                DescFase = ["Projeto" if cont == 0 \
                       else "Fase 1 - Executar MVP" if cont == 1 \
                       else "Fase 2 - Validar MVP" if cont == 2 \
                       else "Fase 3 - Efetivar Mudanças MVP" if cont == 3 \
                       else "Fase 4 - Migrar 72 aplicações" if cont == 4 \
                       else "Fase 5 - Desprovisionar ambiente"]

                IDResp = ["P" if cont == 0 else "G" if cont == 1 else "G" if cont == 2 else "G"]

                InsertFase = "INSERT INTO governanca_cgti_mapa.s_projeto_planejamento_fase (id_fase, id_planejamento, id_projeto, nr_ordem, ds_fase, st_responsavel) \
                              VALUES (" + str(cont+1) + "," + \
                              str(IdPlanejamento) + "," + \
                              str(projSel) + "," + \
                              str(cont) + ",'" + \
                              DescFase[0] + "','" + \
                              IDResp[0] + "')"

                cursorORC.execute(InsertFase)
                conn.commit()
                cont += 1

        else:
            while cont <= 2:
                DescFase = ["Planejamento" if cont == 0 \
                            else "Execução" if cont == 1 \
                            else "Encerramento" if cont == 2 \
                            else "Monitoramento"]

                IDResp = ["P" if cont == 0 else "G" if cont == 1 else "G" if cont == 2 else "G"]

                InsertFase = "INSERT INTO governanca_cgti_mapa.s_projeto_planejamento_fase (id_fase, id_planejamento, id_projeto, nr_ordem, ds_fase, st_responsavel) \
                              VALUES (" + str(cont+1) + "," + \
                              str(IdPlanejamento) + "," + \
                              str(projSel) + "," + str(cont) + ",'" + \
                              DescFase[0] + "','" + \
                              IDResp[0] + "')"

                cursorORC.execute(InsertFase)
                conn.commit()
                cont += 1

        print("Registro Inserido Tabelas FASE Projeto => ", projSel)


    for PastaImp in PastasSel: 
        df = xls.parse(PastaImp)
        x = 0

        print("****** Importando a Pasta =>", PastaImp)

        while(x < len(df)):
            if pd.isnull(df["Unnamed: 5"][x]) == False:

                Prioridade = ["Baixa" if  "Baixa" in df["Unnamed: 5"][x] else "Média" if  "Média" in df["Unnamed: 5"][x] else "Alta" if  "Alta" in df["Unnamed: 5"][x] else "Zero"]

                ServExcel = df["Unnamed: 5"][x].replace("Alta", "").replace("Média", "").replace("Baixa", "").strip()                
                ServExcel = ServExcel.replace("()", "")
                ServExcel = ServExcel.replace("[]", "")

                IdServ, serv = BuscaServico(ServExcel)
                IDTorre, DescTorre = BuscaTorre(df["Unnamed: 4"][x].replace("( ", "(").replace(" )", ")"))

                if df["Unnamed: 4"][x].strip() == "Administrador de Sistemas Operacionais Virtualização Vmware":
                    DescTorre = "Administrador de Sistemas Operac. Virtualização Vmware"
                    IDTorre = "90"           
                if df["Unnamed: 4"][x].strip() == "Administrador de Sistemas Operacionais Linux / Storage":
                    DescTorre = "Administrador de Sistemas Operac.Linux"
                    IDTorre = "91"
                if df["Unnamed: 4"][x].strip() == "Especialista em Redes ( Seguranca )":
                    DescTorre = "Especialista em Redes ( Seguranca )"
                    IDTorre = "26"
                if df["Unnamed: 4"][x].strip() == "Analista de Sistemas de Informação ( Qualidade )":
                    DescTorre = "Analista de Sistemas de Informação ( Qualidade )"
                    IDTorre = "19"
                if df["Unnamed: 4"][x].strip() == "Especialista em Redes ( Microsoft )":
                    DescTorre = "Especialista em Redes ( Microsoft )"
                    IDTorre = "25"
                if df["Unnamed: 4"][x].strip() == "Especialista em Redes ( Backup )":
                    DescTorre = "Especialista em Redes ( Backup )"
                    IDTorre = "24"

                ServExec = df["Unnamed: 9"][x]
                Titulo = ServExcel

                if Container == 0:
                    fase = df["Unnamed: 0"][x]
                    IDFase = [1 if fase == "Projeto" 
                              else 2 if fase == "Fase 1 - Executar MVP" 
                              else 3 if fase == "Fase 2 - Validar MVP" 
                              else 4 if fase == "Fase 3 - Efetivar Mudanças do MVP"
                              else 5 if fase == "Fase 4 - Migrar 72 aplicações"
                              else 6] 
                else:    
                    fase = df["Unnamed: 3"][x]
                    IDFase = [1 if fase == "Fase 1 - Planejamento" else 2 if fase == "Fase 2 - Execução" else 3 if fase == "Fase 4 - Encerramento" else 2]

                if IdServ != "" :
                    IDServSla, SLA, DescServSla, Tempo, Complexidade = BuscaSLA(DescTorre, ServExcel, Prioridade[0])

                if (IdServ) != "":            
                    IDAtividade += 1                 
                    if IDTorre == "":
                        IDTorre = "999999"
                        DescTorre = "Não Encontrado Na Torre"
                    if IdServ == "":
                        IdServ = "999999"
                        serv = "Não Encontrado no Catálogo"

                    InsertAtiv = "INSERT INTO GOVERNANCA_CGTI_MAPA.S_PROJETO_PLANEJAMENTO_ATIVID (ID_ATIVIDADE, ID_PLANEJAMENTO, ID_SERVICO, DS_SERVICO, ID_FILA, NM_FILA, DS_ATIVIDADE, ID_FASE, ID_PROJETO, ID_SLA, DS_SLA, NM_TITULO) \
                                  VALUES (" + str(IDAtividade) + "," + \
                                              str(IdPlanejamento) + "," + \
                                              str(IDServSla) + ",'" + \
                                              str(DescServSla).strip() + "'," + \
                                              str(IDTorre) + ",'" + \
                                              DescTorre.strip() + "','" + \
                                              str(ServExec).strip() + "'," +  \
                                              str(IDFase[0]) + "," + \
                                              str(projSel) + "," + \
                                              str(SLA) + ",'" + \
                                              Prioridade[0] + "','" + \
                                              Titulo.strip() + "')"
                    try:
                        cursorORC.execute(InsertAtiv)
                        conn.commit()
                    except:
                        print("************** Projeto com Planejamento Cadastrado *****************")
                        break
                    else:
                        print(IDAtividade, " - Inclusão da Atividade => Torre - ", DescTorre, " SLA => ", IdServ, " - ", Prioridade[0], " - ", df["Unnamed: 5"][x])
            x += 1   

    print("Fim de Inclusão")
    if con.is_connected():
        cursorMySql.close()
        con.close()
        print("Conexão ao MySQL foi encerrada")
        cursorORC.close()
        conn.close()
        print("Conexão ao ORACLE foi encerrada")

In [11]:
if __name__ == '__main__':
    print("AGUARDE...EM PROCESSAMENTO")
    TelaMain()

AGUARDE...EM PROCESSAMENTO
Conectando aos Bancos
Conectado ao servidor MySQL PRODUÇÃO versão  5.5.64-MariaDB
Conectado ao servidor ORACLE - PRODUÇÃO
Abrindo o PP...
Abrindo o PP...
Abrindo o PP...
Excluindo Planejamento Anterior Para o Projeto..:  1
Abrindo o PP Para o Projeto...:  1
Importando o PP.:  D:/MAPA/Importacao/PP_Sisbravet_Painel_9.xlsx
Registro Inserido Tabelas PLANEJAMENTO Projeto =>  1
Registro Inserido Tabelas FASE Projeto =>  1
****** Importando a Pasta => Planejamento_Previsto
1  - Inclusão da Atividade => Torre -  Especialista de Implementação de Projetos  SLA =>  1021  -  Média  -  Gerencial - Elaborar o Planejamento Preliminar (PP) (Média)
2  - Inclusão da Atividade => Torre -  Analista de Sistemas de Informação (BI)  SLA =>  1335  -  Média  -  Execução - Elaborar documento de origem de dados do painel (Média)
3  - Inclusão da Atividade => Torre -  Analista de Sistemas de Informação (BI)  SLA =>  1335  -  Média  -  Execução - Elaborar documento de origem de dados do

47  - Inclusão da Atividade => Torre -  Analista de Sistemas de Informação (BI)  SLA =>  1330  -  Média  -  Execução - ETL - Elaborar script de Extração de Dados [Média]
48  - Inclusão da Atividade => Torre -  Analista de Sistemas de Informação (BI)  SLA =>  1330  -  Média  -  Execução - ETL - Elaborar script de Extração de Dados [Média]
49  - Inclusão da Atividade => Torre -  Analista de Sistemas de Informação (BI)  SLA =>  1330  -  Média  -  Execução - ETL - Elaborar script de Extração de Dados [Média]
50  - Inclusão da Atividade => Torre -  Analista de Sistemas de Informação (BI)  SLA =>  1330  -  Média  -  Execução - ETL - Elaborar script de Extração de Dados [Média]
51  - Inclusão da Atividade => Torre -  Analista de Sistemas de Informação (BI)  SLA =>  1330  -  Média  -  Execução - ETL - Elaborar script de Extração de Dados [Média]
52  - Inclusão da Atividade => Torre -  Analista de Sistemas de Informação (BI)  SLA =>  1330  -  Média  -  Execução - ETL - Elaborar script de Extraç

99  - Inclusão da Atividade => Torre -  Analista de Sistemas de Informação (BI)  SLA =>  1338  -  Alta  -  Execução - Painel - Criar novo Painel [Alta]
100  - Inclusão da Atividade => Torre -  Analista de Sistemas de Informação (BI)  SLA =>  1338  -  Alta  -  Execução - Painel - Criar novo Painel [Alta]
101  - Inclusão da Atividade => Torre -  Analista de Sistemas de Informação (BI)  SLA =>  1338  -  Alta  -  Execução - Painel - Criar novo Painel [Alta]
102  - Inclusão da Atividade => Torre -  Analista de Sistemas de Informação (BI)  SLA =>  1338  -  Alta  -  Execução - Painel - Criar novo Painel [Alta]
103  - Inclusão da Atividade => Torre -  Analista de Sistemas de Informação (BI)  SLA =>  1338  -  Alta  -  Execução - Painel - Criar novo Painel [Alta]
104  - Inclusão da Atividade => Torre -  Analista de Sistemas de Informação (BI)  SLA =>  1338  -  Média  -  Execução - Painel - Criar novo Painel [Média]
105  - Inclusão da Atividade => Torre -  Analista de Sistemas de Informação (BI) 

153  - Inclusão da Atividade => Torre -  Analista de Sistemas de Informação (BI)  SLA =>  1338  -  Alta  -  Execução - Painel - Criar novo Painel [Alta]
154  - Inclusão da Atividade => Torre -  Analista de Sistemas de Informação (BI)  SLA =>  1338  -  Alta  -  Execução - Painel - Criar novo Painel [Alta]
155  - Inclusão da Atividade => Torre -  Analista de Sistemas de Informação (BI)  SLA =>  1338  -  Alta  -  Execução - Painel - Criar novo Painel [Alta]
156  - Inclusão da Atividade => Torre -  Analista de Sistemas de Informação (BI)  SLA =>  1338  -  Alta  -  Execução - Painel - Criar novo Painel [Alta]
157  - Inclusão da Atividade => Torre -  Analista de Sistemas de Informação (BI)  SLA =>  1338  -  Alta  -  Execução - Painel - Criar novo Painel [Alta]
158  - Inclusão da Atividade => Torre -  Analista de Sistemas de Informação (BI)  SLA =>  1338  -  Alta  -  Execução - Painel - Criar novo Painel [Alta]
159  - Inclusão da Atividade => Torre -  Analista de Sistemas de Informação (BI)  